In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import glob
import os
import datetime
pd.options.display.float_format = '{:,.2f}'.format
datetime.timedelta(days=1)
datetime.timedelta(1)

datetime.timedelta(days=1)

In [13]:
allfiles_adj = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\POLIZAS\2019\10. OCTUBRE\DATA\ADJ*.csv', recursive=False)
allfiles_adj

['C:\\Users\\Cristian.Aparicio\\Documents\\POLIZAS\\2019\\10. OCTUBRE\\DATA\\ADJUSTMENTS OCTUBRE 2019.csv']

In [14]:
allfiles_ret = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\POLIZAS\2019\10. OCTUBRE\DATA\RET*.csv', recursive=False)
allfiles_ret

['C:\\Users\\Cristian.Aparicio\\Documents\\POLIZAS\\2019\\10. OCTUBRE\\DATA\\RETENCIONES OCTUBRE 2019.csv']

In [15]:
allfiles_lib = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\POLIZAS\2019\10. OCTUBRE\DATA\LIB*.csv', recursive=False)
allfiles_lib

['C:\\Users\\Cristian.Aparicio\\Documents\\POLIZAS\\2019\\10. OCTUBRE\\DATA\\LIBERACIONES OCTUBRE 2019.csv']

In [43]:
allfiles_lender = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\POLIZAS\2019\10. OCTUBRE\DATA\LEN*.csv', recursive=True)
allfiles_lender

['C:\\Users\\Cristian.Aparicio\\Documents\\POLIZAS\\2019\\10. OCTUBRE\\DATA\\LENDERS OCTUBRE 2019.csv']

In [19]:
start_date = '2019-10-01'
final_date = '2019-10-31'

# ADJUSTMENTS

In [23]:
adj_1 = pd.concat([pd.read_csv(f).assign(file_name=os.path.basename(f)) for f in allfiles_adj])
adj_1.columns = adj_1.columns.str.strip().str.lower()
adj_1.head()

,report_id,report_date_local,adjustment_date_local,adjustment_type,adjustment_amount,fees_charged,taxes,amount_due,type,date,file_name
0,MXPR78HNW,2019-10-01,2019-10-01,LOAN,-344.97,0.00,0.00,-344.97,DAILY,2019-10-01,ADJUSTMENTS OCTUBRE 2019.csv
1,SPHXMMQ4L,2019-10-01,2019-10-01,LOAN,"-1,650.32",0.00,0.00,"-1,650.32",DAILY,2019-10-01,ADJUSTMENTS OCTUBRE 2019.csv
2,RLC7K2XYQ,2019-10-01,2019-10-01,LOAN,"-2,145.51",0.00,0.00,"-2,145.51",DAILY,2019-10-01,ADJUSTMENTS OCTUBRE 2019.csv
3,3MMGQMJTD,2019-10-01,2019-10-01,LOAN,-74.02,0.00,0.00,-74.02,DAILY,2019-10-01,ADJUSTMENTS OCTUBRE 2019.csv
4,5MXZSVFWR,2019-10-01,2019-10-01,LOAN,-546.82,0.00,0.00,-546.82,DAILY,2019-10-01,ADJUSTMENTS OCTUBRE 2019.csv


In [24]:
adj_1.shape

(1277, 11)

### CURRENT DATES

In [25]:
adj_current_month = adj_1[(adj_1['date'] >= start_date) & (adj_1['date'] <= final_date)]
adj_current_month.head()

,report_id,report_date_local,adjustment_date_local,adjustment_type,adjustment_amount,fees_charged,taxes,amount_due,type,date,file_name
0,MXPR78HNW,2019-10-01,2019-10-01,LOAN,-344.97,0.00,0.00,-344.97,DAILY,2019-10-01,ADJUSTMENTS OCTUBRE 2019.csv
1,SPHXMMQ4L,2019-10-01,2019-10-01,LOAN,"-1,650.32",0.00,0.00,"-1,650.32",DAILY,2019-10-01,ADJUSTMENTS OCTUBRE 2019.csv
2,RLC7K2XYQ,2019-10-01,2019-10-01,LOAN,"-2,145.51",0.00,0.00,"-2,145.51",DAILY,2019-10-01,ADJUSTMENTS OCTUBRE 2019.csv
3,3MMGQMJTD,2019-10-01,2019-10-01,LOAN,-74.02,0.00,0.00,-74.02,DAILY,2019-10-01,ADJUSTMENTS OCTUBRE 2019.csv
4,5MXZSVFWR,2019-10-01,2019-10-01,LOAN,-546.82,0.00,0.00,-546.82,DAILY,2019-10-01,ADJUSTMENTS OCTUBRE 2019.csv


In [26]:
adj_current_month.shape

(1277, 11)

### PIVOT

In [27]:
adj_pivot = adj_current_month.groupby(['type','adjustment_type','date']).agg({'adjustment_amount':sum
                                                                                ,'fees_charged':sum 
                                                                                ,'taxes':sum
                                                                                ,'amount_due':sum}).reset_index()
adj_pivot['adjustment_amount'] = adj_pivot['adjustment_amount']*-1 
adj_pivot['fees_charged'] = adj_pivot['fees_charged']*-1
adj_pivot['taxes'] = adj_pivot['taxes']*-1 
adj_pivot['amount_due'] = adj_pivot['amount_due']*-1 
adj_pivot.head()

,type,adjustment_type,date,adjustment_amount,fees_charged,taxes,amount_due
0,DAILY,CANCELED_TRANSACTION,2019-10-01,"2,158.43",77.65,12.42,"2,068.36"
1,DAILY,CANCELED_TRANSACTION,2019-10-02,"104,744.00","3,770.78",603.33,"100,369.89"
2,DAILY,CANCELED_TRANSACTION,2019-10-03,"83,569.00","3,008.48",481.36,"80,079.16"
3,DAILY,CANCELED_TRANSACTION,2019-10-04,"169,006.64","8,034.74","1,285.56","159,686.34"
4,DAILY,CANCELED_TRANSACTION,2019-10-07,"88,150.00","3,169.95",507.20,"84,472.85"


In [28]:
adj_pivot_v2 = pd.pivot_table(adj_current_month,index=['type','date']
                              ,columns=['adjustment_type']
                              ,values=['adjustment_amount','fees_charged','taxes','amount_due']
                              ,aggfunc=np.sum,margins=True)
adj_pivot_v2.head()

adjustment_amount                                         \
adjustment_type  CANCELED_TRANSACTION ERROR_CORRECTION_ADJUSTMENT       LOAN   
type  date                                                                     
DAILY 2019-10-01            -2,158.43                         NaN -16,980.98   
      2019-10-02          -104,744.00                         NaN -13,647.39   
      2019-10-03           -83,569.00                         NaN -15,625.82   
      2019-10-04          -169,006.64                         NaN -13,629.70   
      2019-10-07           -88,150.00                   -1,762.42 -12,133.40   

                                       amount_due                              \
adjustment_type          All CANCELED_TRANSACTION ERROR_CORRECTION_ADJUSTMENT   
type  date                                                                      
DAILY 2019-10-01  -19,139.41            -2,068.36                         NaN   
      2019-10-02 -118,391.39          -100,369.89                         NaN   
      2019-10-03  -99,194.82           -80,079.16                         NaN   
      2019-10-04 -182,636.34          -159,686.34                         NaN   
      2019-10-07 -102,045.82           -84,472.85                   -1,762.42   

                                                fees_charged  \
adjustment_type        LOAN         All CANCELED_TRANSACTION   
type  date                                                     
DAILY 2019-10-01 -16,980.98  -19,049.34               -77.65   
      2019-10-02 -13,647.39 -114,017.28            -3,770.78   
      2019-10-03 -15,625.82  -95,704.98            -3,008.48   
      2019-10-04 -13,629.70 -173,316.04            -8,034.74   
      2019-10-07 -12,133.40  -98,368.67            -3,169.95   

                                                             \
adjustment_type  ERROR_CORRECTION_ADJUSTMENT LOAN       All   
type  date                                                    
DAILY 2019-10-01                         NaN 0.00    -77.65   
      2019-10-02                         NaN 0.00 -3,770.78   
      2019-10-03                         NaN 0.00 -3,008.48   
      2019-10-04                         NaN 0.00 -8,034.74   
      2019-10-07                        0.00 0.00 -3,169.95   

                                taxes                                   \
adjustment_type  CANCELED_TRANSACTION ERROR_CORRECTION_ADJUSTMENT LOAN   
type  date                                                               
DAILY 2019-10-01               -12.42                         NaN 0.00   
      2019-10-02              -603.33                         NaN 0.00   
      2019-10-03              -481.36                         NaN 0.00   
      2019-10-04            -1,285.56                         NaN 0.00   
      2019-10-07              -507.20                        0.00 0.00   

                            
adjustment_type        All  
type  date                  
DAILY 2019-10-01    -12.42  
      2019-10-02   -603.33  
      2019-10-03   -481.36  
      2019-10-04 -1,285.56  
      2019-10-07   -507.20

# LIBERACIONES

In [29]:
lib_1 = pd.concat([pd.read_csv(f).assign(file_name=os.path.basename(f)) for f in allfiles_lib])
lib_1.columns = lib_1.columns.str.strip().str.lower()
lib_1.head()

,date_release,report_date_local,retained_date_local,report_id,entry_type,amount,type,file_name
0,2019-10-01,2019-10-01,2019-09-02,4PPY578R9,INVALID_BANK_ACCOUNT,"1,696.08",DAILY,LIBERACIONES OCTUBRE 2019.csv
1,2019-10-01,2019-10-01,2019-04-02,LXKRVS2H8,INVALID_BANK_ACCOUNT,70.48,DAILY,LIBERACIONES OCTUBRE 2019.csv
2,2019-10-01,2019-10-01,2019-01-28,LXKRVS2H8,INVALID_BANK_ACCOUNT,165.09,DAILY,LIBERACIONES OCTUBRE 2019.csv
3,2019-10-01,2019-10-01,2019-03-01,LXKRVS2H8,INVALID_BANK_ACCOUNT,27.79,DAILY,LIBERACIONES OCTUBRE 2019.csv
4,2019-10-01,2019-10-01,2019-09-05,LXKRVS2H8,INVALID_BANK_ACCOUNT,149.40,DAILY,LIBERACIONES OCTUBRE 2019.csv


### CURRENT DATES

In [30]:
lib_current_month = lib_1[(lib_1['report_date_local'] >= start_date) & (lib_1['report_date_local'] <= final_date)]
lib_current_month.head()

,date_release,report_date_local,retained_date_local,report_id,entry_type,amount,type,file_name
0,2019-10-01,2019-10-01,2019-09-02,4PPY578R9,INVALID_BANK_ACCOUNT,"1,696.08",DAILY,LIBERACIONES OCTUBRE 2019.csv
1,2019-10-01,2019-10-01,2019-04-02,LXKRVS2H8,INVALID_BANK_ACCOUNT,70.48,DAILY,LIBERACIONES OCTUBRE 2019.csv
2,2019-10-01,2019-10-01,2019-01-28,LXKRVS2H8,INVALID_BANK_ACCOUNT,165.09,DAILY,LIBERACIONES OCTUBRE 2019.csv
3,2019-10-01,2019-10-01,2019-03-01,LXKRVS2H8,INVALID_BANK_ACCOUNT,27.79,DAILY,LIBERACIONES OCTUBRE 2019.csv
4,2019-10-01,2019-10-01,2019-09-05,LXKRVS2H8,INVALID_BANK_ACCOUNT,149.40,DAILY,LIBERACIONES OCTUBRE 2019.csv


In [31]:
lib_current_month.shape

(30289, 8)

### PIVOT

In [32]:
lib_pivot = lib_current_month.groupby(['type','entry_type','report_date_local']).agg({'amount':sum}).reset_index()
lib_pivot.head()

,type,entry_type,report_date_local,amount
0,DAILY,INVALID_BANK_ACCOUNT,2019-10-01,"886,749.88"
1,DAILY,INVALID_BANK_ACCOUNT,2019-10-02,"924,207.23"
2,DAILY,INVALID_BANK_ACCOUNT,2019-10-03,"1,015,059.25"
3,DAILY,INVALID_BANK_ACCOUNT,2019-10-04,"620,803.91"
4,DAILY,INVALID_BANK_ACCOUNT,2019-10-07,"1,060,172.12"


In [33]:
lib_pivot.shape

(46, 4)

In [34]:
lib_pivot_v2 = pd.pivot_table(lib_current_month,index=['type','report_date_local']
                              ,columns=['entry_type']
                              ,values=['amount']
                              ,aggfunc=np.sum,margins=True)
lib_pivot_v2.head()

amount                          
entry_type              INVALID_BANK_ACCOUNT         LIEN          All
type  report_date_local                                               
DAILY 2019-10-01                  886,749.88   312,282.33 1,199,032.21
      2019-10-02                  924,207.23 1,281,671.83 2,205,879.06
      2019-10-03                1,015,059.25 1,341,990.91 2,357,050.16
      2019-10-04                  620,803.91   697,366.29 1,318,170.20
      2019-10-07                1,060,172.12 1,003,149.18 2,063,321.30

In [35]:
lib_pivot_v2.shape

(24, 3)

# RETENCIONES

In [36]:
ret_1 = pd.concat([pd.read_csv(f).assign(file_name=os.path.basename(f)) for f in allfiles_ret])
ret_1.columns = ret_1.columns.str.strip().str.lower()
ret_1.head()

,date_retainment,report_date_local,report_id,entry_type,amount,type,file_name
0,2019-10-01,2019-10-01,SJXZWV9YY,INVALID_BANK_ACCOUNT,"12,523.86",DAILY,RETENCIONES OCTUBRE 2019.csv
1,2019-10-01,2019-10-01,MW4W4V6LH,INVALID_BANK_ACCOUNT,"36,087.56",DAILY,RETENCIONES OCTUBRE 2019.csv
2,2019-10-01,2019-10-01,5NKTS6DF3,INVALID_BANK_ACCOUNT,"4,702.73",DAILY,RETENCIONES OCTUBRE 2019.csv
3,2019-10-01,2019-10-01,MSTK6C5DN,LIEN,"16,002.61",DAILY,RETENCIONES OCTUBRE 2019.csv
4,2019-10-01,2019-10-01,765WY5JBK,LIEN,"27,214.02",DAILY,RETENCIONES OCTUBRE 2019.csv


### CURRENT DATES

In [37]:
ret_current_month = ret_1[(ret_1['report_date_local'] >= start_date) & (ret_1['report_date_local'] <= final_date)]
ret_current_month.head()

,date_retainment,report_date_local,report_id,entry_type,amount,type,file_name
0,2019-10-01,2019-10-01,SJXZWV9YY,INVALID_BANK_ACCOUNT,"12,523.86",DAILY,RETENCIONES OCTUBRE 2019.csv
1,2019-10-01,2019-10-01,MW4W4V6LH,INVALID_BANK_ACCOUNT,"36,087.56",DAILY,RETENCIONES OCTUBRE 2019.csv
2,2019-10-01,2019-10-01,5NKTS6DF3,INVALID_BANK_ACCOUNT,"4,702.73",DAILY,RETENCIONES OCTUBRE 2019.csv
3,2019-10-01,2019-10-01,MSTK6C5DN,LIEN,"16,002.61",DAILY,RETENCIONES OCTUBRE 2019.csv
4,2019-10-01,2019-10-01,765WY5JBK,LIEN,"27,214.02",DAILY,RETENCIONES OCTUBRE 2019.csv


In [38]:
ret_current_month.shape

(2866, 7)

### PIVOT

In [39]:
ret_pivot = ret_current_month.groupby(['type','entry_type','report_date_local']).agg({'amount':sum}).reset_index()
ret_pivot.head()

,type,entry_type,report_date_local,amount
0,DAILY,INVALID_BANK_ACCOUNT,2019-10-01,"935,296.11"
1,DAILY,INVALID_BANK_ACCOUNT,2019-10-02,"914,965.58"
2,DAILY,INVALID_BANK_ACCOUNT,2019-10-03,"1,006,881.90"
3,DAILY,INVALID_BANK_ACCOUNT,2019-10-04,"615,460.32"
4,DAILY,INVALID_BANK_ACCOUNT,2019-10-07,"1,142,268.38"


In [40]:
ret_pivot.shape

(46, 4)

In [41]:
ret_pivot_v2 = pd.pivot_table(ret_current_month,index=['type','report_date_local']
                              ,columns=['entry_type']
                              ,values=['amount']
                              ,aggfunc=np.sum,margins=True)
ret_pivot_v2.head()

amount                          
entry_type              INVALID_BANK_ACCOUNT         LIEN          All
type  report_date_local                                               
DAILY 2019-10-01                  935,296.11 1,187,314.60 2,122,610.71
      2019-10-02                  914,965.58   813,110.49 1,728,076.07
      2019-10-03                1,006,881.90 1,246,231.80 2,253,113.70
      2019-10-04                  615,460.32 1,127,176.33 1,742,636.65
      2019-10-07                1,142,268.38 1,908,294.07 3,050,562.45

In [42]:
ret_pivot_v2.shape

(24, 3)

# LENDERS

In [44]:
lender_1 = pd.concat([pd.read_csv(f).assign(file_name=os.path.basename(f)) for f in allfiles_lender])
lender_1.columns = lender_1.columns.str.strip().str.lower()
lender_1.head()

,collection_date_local,loan_id,provider_code,collection_origin,source,amount,capital,interest,tax,file_name
0,2019-10-01,0f8344d8-1431-47e3-914f-e921bd5a7b42,CLIP,SETTLEMENT,NaN,185.42,157.94,23.69,3.79,LENDERS OCTUBRE 2019.csv
1,2019-10-02,0f8344d8-1431-47e3-914f-e921bd5a7b42,CLIP,SETTLEMENT,NaN,43.12,36.73,5.51,0.88,LENDERS OCTUBRE 2019.csv
2,2019-10-03,0f8344d8-1431-47e3-914f-e921bd5a7b42,CLIP,SETTLEMENT,NaN,100.62,85.70,12.86,2.06,LENDERS OCTUBRE 2019.csv
3,2019-10-04,0f8344d8-1431-47e3-914f-e921bd5a7b42,CLIP,SETTLEMENT,NaN,150.93,128.56,19.28,3.09,LENDERS OCTUBRE 2019.csv
4,2019-10-07,0f8344d8-1431-47e3-914f-e921bd5a7b42,CLIP,SETTLEMENT,NaN,238.60,203.23,30.49,4.88,LENDERS OCTUBRE 2019.csv


In [45]:
lender_1.shape

(1054, 10)

### CURRENT DATES

In [47]:
len_current_month = lender_1[(lender_1['collection_date_local'] >= start_date) & (lender_1['collection_date_local'] <= final_date)]
len_current_month.head()

,collection_date_local,loan_id,provider_code,collection_origin,source,amount,capital,interest,tax,file_name
0,2019-10-01,0f8344d8-1431-47e3-914f-e921bd5a7b42,CLIP,SETTLEMENT,NaN,185.42,157.94,23.69,3.79,LENDERS OCTUBRE 2019.csv
1,2019-10-02,0f8344d8-1431-47e3-914f-e921bd5a7b42,CLIP,SETTLEMENT,NaN,43.12,36.73,5.51,0.88,LENDERS OCTUBRE 2019.csv
2,2019-10-03,0f8344d8-1431-47e3-914f-e921bd5a7b42,CLIP,SETTLEMENT,NaN,100.62,85.70,12.86,2.06,LENDERS OCTUBRE 2019.csv
3,2019-10-04,0f8344d8-1431-47e3-914f-e921bd5a7b42,CLIP,SETTLEMENT,NaN,150.93,128.56,19.28,3.09,LENDERS OCTUBRE 2019.csv
4,2019-10-07,0f8344d8-1431-47e3-914f-e921bd5a7b42,CLIP,SETTLEMENT,NaN,238.60,203.23,30.49,4.88,LENDERS OCTUBRE 2019.csv


In [48]:
len_current_month.shape

(1054, 10)

### PIVOT

In [50]:
len_pivot = len_current_month.groupby(['provider_code','collection_date_local']).agg({'amount':sum,'capital':sum,
                                                                                     'interest':sum,'tax':sum}).reset_index()
len_pivot.head()

,provider_code,collection_date_local,amount,capital,interest,tax
0,CLIP,2019-10-01,"1,136.47",968.03,145.22,23.22
1,CLIP,2019-10-02,"1,000.00",851.79,127.76,20.45
2,CLIP,2019-10-03,689.63,587.41,88.13,14.09
3,CLIP,2019-10-04,897.87,764.79,114.72,18.36
4,CLIP,2019-10-07,"2,627.34","2,237.92",335.70,53.72


In [51]:
len_pivot.shape

(46, 6)

# EXCEL FINAL

In [52]:
writer = pd.ExcelWriter('FASE II. DATA POLIZA OCTUBRE 2019.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
len_pivot.to_excel(writer, sheet_name='DATA LENDERS',index=False)
adj_pivot.to_excel(writer, sheet_name='DATA ADJUSTMENTS',index=False)
lib_pivot_v2.to_excel(writer, sheet_name='DATA LIBERACIONES')
ret_pivot_v2.to_excel(writer, sheet_name='DATA RETENCIONES')
writer.close()